In [ ]:
import nannyml as nml
from IPython.display import display

reference_df = nml.load_synthetic_binary_classification_dataset()[0]
analysis_df = nml.load_synthetic_binary_classification_dataset()[1]
display(reference_df.head())

,distance_from_office,salary_range,gas_price_per_litre,public_transportation_cost,wfh_prev_workday,workday,tenure,identifier,work_home_actual,timestamp,y_pred_proba,period,y_pred
0,5.962247,40K - 60K €,2.119485,8.568058,False,Friday,0.212653,0,1,2014-05-09 22:27:20,0.99,reference,1
1,0.535872,40K - 60K €,2.357199,5.425382,True,Tuesday,4.927549,1,0,2014-05-09 22:59:32,0.07,reference,0
2,1.969519,40K - 60K €,2.366849,8.247158,False,Monday,0.520817,2,1,2014-05-09 23:48:25,1.00,reference,1
3,2.530410,20K - 40K €,2.318722,7.944251,False,Tuesday,0.453649,3,1,2014-05-10 01:12:09,0.98,reference,1
4,2.253635,60K+ €,2.221265,8.884478,True,Thursday,5.695263,4,1,2014-05-10 02:21:34,0.99,reference,1


In [ ]:
print(reference_df.head().to_markdown(tablefmt="grid"))

+----+------------------------+----------------+-----------------------+------------------------------+--------------------+-----------+----------+--------------+--------------------+---------------------+----------------+-----------+----------+
|    |   distance_from_office | salary_range   |   gas_price_per_litre |   public_transportation_cost | wfh_prev_workday   | workday   |   tenure |   identifier |   work_home_actual | timestamp           |   y_pred_proba | period    |   y_pred |
+====+========================+================+=======================+==============================+====================+===========+==========+==============+====================+=====================+================+===========+==========+
|  0 |               5.96225  | 40K - 60K €    |               2.11948 |                      8.56806 | False              | Friday    | 0.212653 |            0 |                  1 | 2014-05-09 22:27:20 |           0.99 | reference |        1 |
+----+----------

In [ ]:
# explicitly set the y_pred column as categorical.
# if left as is, it would be designated as continuous because of the int dtype
reference_df['y_pred'] = reference_df['y_pred'].astype("category")
analysis_df['y_pred'] = analysis_df['y_pred'].astype("category")

column_names = ['distance_from_office', 'salary_range', 'gas_price_per_litre', 'public_transportation_cost', 'wfh_prev_workday', 'workday', 'tenure', 'y_pred_proba', 'y_pred']
calc = nml.UnivariateDriftCalculator(
    column_names=column_names,
    timestamp_column_name='timestamp',
    continuous_methods=['kolmogorov_smirnov', 'jensen_shannon'],
    categorical_methods=['chi2', 'jensen_shannon'],
)

In [ ]:
calc.fit(reference_df)
results = calc.calculate(analysis_df)
display(results.filter(period='analysis', column_names=['distance_from_office']).to_df())

chunk                                                         \
        chunk                                                          
  chunk_index            end_date end_index            key    period   
0           0 2018-01-02 00:45:44      4999       [0:4999]  analysis   
1           1 2018-05-01 13:10:10      9999    [5000:9999]  analysis   
2           2 2018-09-01 15:40:40     14999  [10000:14999]  analysis   
3           3 2018-12-31 10:11:21     19999  [15000:19999]  analysis   
4           4 2019-04-30 11:01:30     24999  [20000:24999]  analysis   
5           5 2019-09-01 00:24:27     29999  [25000:29999]  analysis   
6           6 2019-12-31 09:09:12     34999  [30000:34999]  analysis   
7           7 2020-04-30 11:46:53     39999  [35000:39999]  analysis   
8           8 2020-09-01 02:46:02     44999  [40000:44999]  analysis   
9           9 2021-01-01 04:29:32     49999  [45000:49999]  analysis   

                                  distance_from_office                  \
                                    kolmogorov_smirnov                   
           start_date start_index                alert lower_threshold   
0 2017-08-31 04:20:00           0                False            None   
1 2018-01-02 01:13:11        5000                False            None   
2 2018-05-01 14:25:25       10000                False            None   
3 2018-09-01 16:19:07       15000                False            None   
4 2018-12-31 10:38:45       20000                False            None   
5 2019-04-30 11:02:00       25000                 True            None   
6 2019-09-01 00:28:54       30000                 True            None   
7 2019-12-31 10:07:15       35000                 True            None   
8 2020-04-30 12:04:32       40000                 True            None   
9 2020-09-01 02:46:13       45000                 True            None   

                                                                           \
                           jensen_shannon                                   
  upper_threshold    value          alert lower_threshold upper_threshold   
0            None  0.01310          False            None             0.1   
1            None  0.01124          False            None             0.1   
2            None  0.01682          False            None             0.1   
3            None  0.01436          False            None             0.1   
4            None  0.01116          False            None             0.1   
5            None  0.43548           True            None             0.1   
6            None  0.43032           True            None             0.1   
7            None  0.43786           True            None             0.1   
8            None  0.43608           True            None             0.1   
9            None  0.43852           True            None             0.1   

             
             
      value  
0  0.026101  
1  0.020297  
2  0.021096  
3  0.036210  
4  0.028708  
5  0.464732  
6  0.460044  
7  0.466746  
8  0.466300  
9  0.467798

In [ ]:
print(results.filter(period='analysis', column_names=['distance_from_office']).to_df().to_markdown(tablefmt="grid"))

+----+-------------------------------------+----------------------------------+-----------------------------------+-----------------------------+--------------------------------+------------------------------------+-------------------------------------+-----------------------------------------------------------+---------------------------------------------------------------------+---------------------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------+-----------------------------------------------------------------+-----------------------------------------------------------------+-------------------------------------------------------+
|    |   ('chunk', 'chunk', 'chunk_index') | ('chunk', 'chunk', 'end_date')   |   ('chunk', 'chunk', 'end_index') | ('chunk', 'chunk', 'key')   | ('chunk', 'chunk', 'period')   | ('chunk', 'chunk', 'start_date')   |   ('chunk', 'chunk', 'star

In [ ]:
display(results.filter(period='analysis', column_names=['distance_from_office']).to_df(multilevel=False))

,chunk_index,chunk_end_date,chunk_end_index,chunk_key,chunk_period,chunk_start_date,chunk_start_index,distance_from_office_kolmogorov_smirnov_alert,distance_from_office_kolmogorov_smirnov_lower_threshold,distance_from_office_kolmogorov_smirnov_upper_threshold,distance_from_office_kolmogorov_smirnov_value,distance_from_office_jensen_shannon_alert,distance_from_office_jensen_shannon_lower_threshold,distance_from_office_jensen_shannon_upper_threshold,distance_from_office_jensen_shannon_value
0,0,2018-01-02 00:45:44,4999,[0:4999],analysis,2017-08-31 04:20:00,0,False,None,None,0.01310,False,None,0.1,0.026101
1,1,2018-05-01 13:10:10,9999,[5000:9999],analysis,2018-01-02 01:13:11,5000,False,None,None,0.01124,False,None,0.1,0.020297
2,2,2018-09-01 15:40:40,14999,[10000:14999],analysis,2018-05-01 14:25:25,10000,False,None,None,0.01682,False,None,0.1,0.021096
3,3,2018-12-31 10:11:21,19999,[15000:19999],analysis,2018-09-01 16:19:07,15000,False,None,None,0.01436,False,None,0.1,0.036210
4,4,2019-04-30 11:01:30,24999,[20000:24999],analysis,2018-12-31 10:38:45,20000,False,None,None,0.01116,False,None,0.1,0.028708
5,5,2019-09-01 00:24:27,29999,[25000:29999],analysis,2019-04-30 11:02:00,25000,True,None,None,0.43548,True,None,0.1,0.464732
6,6,2019-12-31 09:09:12,34999,[30000:34999],analysis,2019-09-01 00:28:54,30000,True,None,None,0.43032,True,None,0.1,0.460044
7,7,2020-04-30 11:46:53,39999,[35000:39999],analysis,2019-12-31 10:07:15,35000,True,None,None,0.43786,True,None,0.1,0.466746
8,8,2020-09-01 02:46:02,44999,[40000:44999],analysis,2020-04-30 12:04:32,40000,True,None,None,0.43608,True,None,0.1,0.466300
9,9,2021-01-01 04:29:32,49999,[45000:49999],analysis,2020-09-01 02:46:13,45000,True,None,None,0.43852,True,None,0.1,0.467798


In [ ]:
print(results.filter(period='analysis', column_names=['distance_from_office']).to_df(multilevel=False).to_markdown(tablefmt="grid"))

+----+---------------+---------------------+-------------------+---------------+----------------+---------------------+---------------------+-------------------------------------------------+-----------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------+---------------------------------------------+-------------------------------------------------------+-------------------------------------------------------+---------------------------------------------+
|    |   chunk_index | chunk_end_date      |   chunk_end_index | chunk_key     | chunk_period   | chunk_start_date    |   chunk_start_index | distance_from_office_kolmogorov_smirnov_alert   | distance_from_office_kolmogorov_smirnov_lower_threshold   | distance_from_office_kolmogorov_smirnov_upper_threshold   |   distance_from_office_kolmogorov_smirnov_value | distance_from_office_jensen_shannon_alert   | distance_from_office_jense

In [ ]:
display(results.filter(period='reference', column_names=['distance_from_office']).to_df())

chunk                                                          \
        chunk                                                           
  chunk_index            end_date end_index            key     period   
0           0 2014-09-09 08:18:27      4999       [0:4999]  reference   
1           1 2015-01-09 00:02:51      9999    [5000:9999]  reference   
2           2 2015-05-09 15:54:26     14999  [10000:14999]  reference   
3           3 2015-09-07 07:14:37     19999  [15000:19999]  reference   
4           4 2016-01-08 16:02:05     24999  [20000:24999]  reference   
5           5 2016-05-09 11:09:39     29999  [25000:29999]  reference   
6           6 2016-09-04 03:30:35     34999  [30000:34999]  reference   
7           7 2017-01-03 18:48:21     39999  [35000:39999]  reference   
8           8 2017-05-03 02:34:24     44999  [40000:44999]  reference   
9           9 2017-08-31 03:10:29     49999  [45000:49999]  reference   

                                  distance_from_office                  \
                                    kolmogorov_smirnov                   
           start_date start_index                alert lower_threshold   
0 2014-05-09 22:27:20           0                False            None   
1 2014-09-09 09:13:35        5000                False            None   
2 2015-01-09 00:04:43       10000                False            None   
3 2015-05-09 16:02:08       15000                False            None   
4 2015-09-07 07:27:47       20000                False            None   
5 2016-01-08 17:22:00       25000                False            None   
6 2016-05-09 11:19:36       30000                False            None   
7 2016-09-04 04:09:35       35000                False            None   
8 2017-01-03 19:00:51       40000                False            None   
9 2017-05-03 02:49:38       45000                False            None   

                                                                           \
                           jensen_shannon                                   
  upper_threshold    value          alert lower_threshold upper_threshold   
0            None  0.01034          False            None             0.1   
1            None  0.00750          False            None             0.1   
2            None  0.00820          False            None             0.1   
3            None  0.00860          False            None             0.1   
4            None  0.00910          False            None             0.1   
5            None  0.01458          False            None             0.1   
6            None  0.01290          False            None             0.1   
7            None  0.01380          False            None             0.1   
8            None  0.01586          False            None             0.1   
9            None  0.00924          False            None             0.1   

             
             
      value  
0  0.029464  
1  0.023659  
2  0.026440  
3  0.021773  
4  0.023972  
5  0.027577  
6  0.026875  
7  0.031265  
8  0.027352  
9  0.029627

In [ ]:
print(results.filter(period='reference', column_names=['distance_from_office']).to_df().to_markdown(tablefmt="grid"))

+----+-------------------------------------+----------------------------------+-----------------------------------+-----------------------------+--------------------------------+------------------------------------+-------------------------------------+-----------------------------------------------------------+---------------------------------------------------------------------+---------------------------------------------------------------------+-----------------------------------------------------------+-------------------------------------------------------+-----------------------------------------------------------------+-----------------------------------------------------------------+-------------------------------------------------------+
|    |   ('chunk', 'chunk', 'chunk_index') | ('chunk', 'chunk', 'end_date')   |   ('chunk', 'chunk', 'end_index') | ('chunk', 'chunk', 'key')   | ('chunk', 'chunk', 'period')   | ('chunk', 'chunk', 'start_date')   |   ('chunk', 'chunk', 'star

In [ ]:
drift_fig = results.filter(column_names=results.continuous_column_names, methods=['jensen_shannon']).plot(kind='drift')
drift_fig.show()

In [ ]:
drift_fig = results.filter(column_names=results.continuous_column_names, methods=['jensen_shannon']).plot(kind='drift')
drift_fig.write_image(f'../_static/drift-guide-continuous.svg')

In [ ]:
drift_fig = results.filter(column_names=results.categorical_column_names, methods=['chi2']).plot(kind='drift')
drift_fig.show()

In [ ]:
drift_fig = results.filter(column_names=results.categorical_column_names, methods=['chi2']).plot(kind='drift')
drift_fig.write_image(f'../_static/drift-guide-categorical.svg')

In [ ]:
figure = results.filter(column_names=results.continuous_column_names, methods=['jensen_shannon']).plot(kind='distribution')
figure.show()

In [ ]:
figure = results.filter(column_names=results.continuous_column_names, methods=['jensen_shannon']).plot(kind='distribution')
figure.write_image(f'../_static/drift-guide-joyplot-continuous.svg')

In [ ]:
figure = results.filter(column_names=results.categorical_column_names, methods=['chi2']).plot(kind='distribution')
figure.show()

In [ ]:
figure = results.filter(column_names=results.categorical_column_names, methods=['chi2']).plot(kind='distribution')
figure.write_image(f'../_static/drift-guide-stacked-categorical.svg')

In [ ]:
ranker = nml.AlertCountRanker()
ranked_features = ranker.rank(
    results.filter(methods=['chi2', 'kolmogorov_smirnov']),
    only_drifting = False
)
display(ranked_features)

,number_of_alerts,column_name,rank
0,6,y_pred_proba,1
1,5,y_pred,2
2,5,wfh_prev_workday,3
3,5,salary_range,4
4,5,public_transportation_cost,5
5,5,distance_from_office,6
6,2,tenure,7
7,0,workday,8
8,0,gas_price_per_litre,9


In [ ]:
print(ranked_features.to_markdown(tablefmt="grid"))


+----+--------------------+----------------------------+--------+
|    |   number_of_alerts | column_name                |   rank |
+====+====================+============================+========+
|  0 |                  6 | y_pred_proba               |      1 |
+----+--------------------+----------------------------+--------+
|  1 |                  5 | y_pred                     |      2 |
+----+--------------------+----------------------------+--------+
|  2 |                  5 | wfh_prev_workday           |      3 |
+----+--------------------+----------------------------+--------+
|  3 |                  5 | salary_range               |      4 |
+----+--------------------+----------------------------+--------+
|  4 |                  5 | public_transportation_cost |      5 |
+----+--------------------+----------------------------+--------+
|  5 |                  5 | distance_from_office       |      6 |
+----+--------------------+----------------------------+--------+
|  6 |    